# 📈 Fetch and Clean Dataset: F3 Innovate Frost Risk Forecasting
- **Converted File to Df** so I could use data
- **Dictionary of Name -> Data** such that each station name maps to it's historic data

In [ ]:
!pip install pandas # If Pandas is not already Installed
!pip install matplotlib

In [4]:
import pandas as pd 

DATA_DIR = "../cimis-hourly-data-multiple-stations/"
def get_datasets() -> dict: 
    """
        Get Dictionary of All (Key (Station-Name), Value (DataFrame)) pairs
    """
    !touch buffer.tmp
    !(ls $DATA_DIR | grep "csv") > buffer.tmp
    name_to_data = dict() # Dictionary for Name to Data
    # Read the buffer.tmp and create Name to Data pairs
    with open("buffer.tmp", "r") as file:
        contents = [f for f in file.read().split("\n") if f]
        for content in contents:
            file_name = content.split(".")[0]
            if "all" not in file_name:
                name_to_data[file_name] = pd.read_csv(DATA_DIR + content)
            
        !rm -f buffer.tmp
        return name_to_data

def clean_datasets(datasets: dict) -> dict:
    """
        Clean the Dictionary of Datasets
    """
    for _, df in datasets.items():
        df.drop(columns=[col for col in df.columns if "qc" in col], inplace=True)
        df.rename(columns=lambda c: c.replace(" ", "_"), inplace=True)
    return datasets

In [5]:
datasets = clean_datasets(get_datasets()) # Grabbed and Cleaned Datasets

# 🔦 LightGBM Integration: Calibrated Frost Probabilities + Temperature Forecasts
- **Calibrated classification** models for frost within 3/6/12/24 hours (Isotonic).
- **Regression** models (and optional quantiles) for temperature at +H hours.
- Utilities that respect time ordering (no leakage).

In [ ]:
# If LightGBM isn't installed in your environment, uncomment and run:
!pip install lightgbm
!pip install scikit-learn

In [6]:
import numpy as np, pandas as pd
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    brier_score_loss, roc_auc_score, average_precision_score, 
    mean_absolute_error, confusion_matrix
)
import lightgbm as lgb

def make_features(df, H):
    """
    Make new Features for the Model
    NOTE: Spaces (" ") in Original Features are Replaced with underscores ("_")
    
    Compromise behavior:
      - If df has a single station -> use plain rolling/diff (no groupby).
      - If df has multiple stations -> use groupby("Stn_Id") for rolling/diff.
    """
    df = df.copy()

    # --------- Detect single-station vs multi-station mode ---------
    has_stn = "Stn_Id" in df.columns
    n_stations = df["Stn_Id"].nunique() if has_stn else 1
    multi_station = has_stn and n_stations > 1

    if multi_station:
        # Multi-station: groupby
        g = df.groupby("Stn_Id", group_keys=False)

        def roll(col, w):
            return (
                g[col]
                .rolling(w)
                .min()
                .reset_index(level=0, drop=True)
            )

        def diff(col, w):
            return g[col].diff(w)

    else:
        # Single-station: no groupby needed
        
        def roll(col, w):
            return df[col].rolling(w).min()

        def diff(col, w):
            return df[col].diff(w)

    # ---------- Simple per-row features (no groupby needed) ----------
    # Dewpoint depression: closer = more humid
    df["dewpoint_dep"] = df["Air_Temp_(C)"] - df["Dew_Point_(C)"]

    # Is dim sunlight
    df["is_dim"] = (df["Sol_Rad_(W/sq.m)"] < 50).astype(int)

    # Is very calm winds
    df["is_very_calm"] = (df["Wind_Speed_(m/s)"] < 1.0).astype(int)

    # Is winter based on Julian day
    jd = pd.to_numeric(df["Jul"], errors="coerce").fillna(0)  # Julian Day
    WINTER_START = 305   # Nov 1
    WINTER_END   =  74   # Mar 15
    df["is_winter"] = ((jd >= WINTER_START) | (jd <= WINTER_END)).astype(int)

    # Hour/day cyclical encodings
    hour = pd.to_numeric(df["Hour_(PST)"], errors="coerce").fillna(0) % 24
    hour_angle = 2 * np.pi * hour / 24.0

    day = pd.to_numeric(df["Jul"], errors="coerce").fillna(0) % 365
    day_angle = 2 * np.pi * day / 365.0

    df["hour_sin"] = np.sin(hour_angle)
    df["hour_cos"] = np.cos(hour_angle)
    df["day_sin"] = np.sin(day_angle)
    df["day_cos"] = np.cos(day_angle)

    # ---------- Time-series features (roll/diff via helpers) ----------
    tfs = [3, 6, 12, 24]
    for time in tfs:
        # Air Temperature
        df[f"air_temp_roll_min_{time}h"] = roll("Air_Temp_(C)", time)
        df[f"air_temp_change_{time}h"]   = diff("Air_Temp_(C)", time)

        # Dew Point
        df[f"dew_point_roll_min_{time}h"] = roll("Dew_Point_(C)", time)
        df[f"dew_point_change_{time}h"]   = diff("Dew_Point_(C)", time)

        # Wind Speed
        df[f"wind_spd_roll_min_{time}h"] = roll("Wind_Speed_(m/s)", time)
        df[f"wind_spd_change_{time}h"]   = diff("Wind_Speed_(m/s)", time)

        # Solar Radiation
        df[f"sol_rad_roll_min_{time}h"] = roll("Sol_Rad_(W/sq.m)", time)
        df[f"sol_rad_change_{time}h"]   = diff("Sol_Rad_(W/sq.m)", time)

    return df


def add_targets(df, H):
    """
        Make Targets for the Model
        NOTE: Spaces (" ") in Original Features are Replaced with underscores ("_")
    """
    df[f"frost_{H}h"] = (df["Air_Temp_(C)"].shift(-H) < 0).astype(int)
    df[f"temp_{H}h"]  =  df["Air_Temp_(C)"].shift(-H)
    return df

def build_xy(df_raw, H, feature_cols=None):
    """
        Build the X (Features) and Y (Targets) for the Model
    """
    # Build rebuild df
    df = df_raw.copy()
    df = make_features(df, H)
    df = add_targets(df, H)
    
    # Targets
    frost_col = f"frost_{H}h"
    temp_col  = f"temp_{H}h"
    # Columns to Ignore
    ignore_cols = ['Stn_Id','Stn_Name','CIMIS_Region', 'Date', 'Hour_(PST)', 'Jul']

    # Filter for Feature Cols
    if feature_cols is None:
        is_valid_feature_col = lambda c: c in df.columns and c not in [frost_col, temp_col, *ignore_cols]
        feature_cols = [c for c in df.columns if is_valid_feature_col(c)]

    # X (Features) and Y (Targets)
    needed = feature_cols + [frost_col, temp_col]
    df = df.dropna(subset=[c for c in needed if c in df.columns])

    X = df[feature_cols]
    y_frost = df[frost_col].astype(int)
    y_temp  = df[temp_col].astype(float)
    return X, y_frost, y_temp, feature_cols



In [7]:
import matplotlib.pyplot as plt
import numpy as np

def compute_and_plot_feature_corr(df_raw, H=6, feature_cols=None, figsize=(10,8)):
    """
    Build X using your pipeline, compute feature↔feature correlation matrix,
    and plot it using matplotlib.
    """
    # Build features only (no targets in matrix)
    X, _, _, used_features = build_xy(df_raw, H, feature_cols=feature_cols)
    # Correlation matrix
    corr = X.corr(method="pearson")

    # --- Plot ---
    plt.figure(figsize=figsize)
    plt.imshow(corr, cmap='coolwarm', interpolation='nearest')
    plt.colorbar(label='Correlation')

    # Tick labels
    plt.xticks(ticks=np.arange(len(used_features)), labels=used_features, rotation=90)
    plt.yticks(ticks=np.arange(len(used_features)), labels=used_features)

    plt.title(f"Feature ↔ Feature Correlation Matrix (H={H}h)")
    plt.tight_layout()
    plt.show()

    return corr

name = "80-fresnostate"
df = datasets[name]
#compute_and_plot_feature_corr(df)

In [8]:
def train_model(df, H=6, threshold=0.5, verbose=True):
    # Build features/targets with your existing helpers
    X, y_frost, y_temp, feat_cols = build_xy(df, H)  # uses ensure_timestamp -> make_features -> add_targets

    # Time-ordered split: 70% train, 15% calibration, 15% test
    n = len(X)
    i_tr, i_cal = int(n * 0.70), int(n * 0.85)

    X_tr, y_tr = X.iloc[:i_tr], y_frost.iloc[:i_tr]
    X_cal, y_cal = X.iloc[i_tr:i_cal], y_frost.iloc[i_tr:i_cal]
    X_te,  y_te  = X.iloc[i_cal:],      y_frost.iloc[i_cal:]
    y_temp_trcal = y_temp.iloc[:i_cal]
    y_temp_te    = y_temp.iloc[i_cal:]

    # --- Classifier ---
    clf = lgb.LGBMClassifier(
        n_estimators=800,
        learning_rate=0.002,
        class_weight="balanced",
        subsample=0.8,
        colsample_bytree=0.9,
        random_state=42,
        verbose=-1
    )
    clf.fit(X_tr, y_tr)

    # Probability calibration on the calibration window (avoids cv='prefit' deprecation)
    calib = CalibratedClassifierCV(estimator=clf, method="isotonic", cv=5)
    calib.fit(X_cal, y_cal)
    p_te = calib.predict_proba(X_te)[:, 1]

    # --- Turn probabilities into hard predictions at the chosen threshold ---
    y_pred = (p_te >= threshold).astype(int)

    # Confusion matrix: TN, FP, FN, TP
    tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()

    # Derived stats
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    fpr       = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    tnr       = tn / (fp + tn) if (fp + tn) > 0 else 0.0

    # --- Regressor (train on train+cal) ---
    reg = lgb.LGBMRegressor(
        n_estimators=400,
        learning_rate=0.04,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42
    )
    reg.fit(pd.concat([X_tr, X_cal]), y_temp_trcal)
    yhat_te = reg.predict(X_te)

    # --- Metrics ---
    metrics = {
        "AUROC":    float(roc_auc_score(y_te, p_te)),
        "AUPRC":    float(average_precision_score(y_te, p_te)),
        "Brier":    float(brier_score_loss(y_te, p_te)),
        "MAE_temp": float(mean_absolute_error(y_temp_te, yhat_te)),
        "threshold": float(threshold),
        "TP": int(tp),
        "FP": int(fp),
        "TN": int(tn),
        "FN": int(fn),
        "precision": float(precision),
        "recall": float(recall),
        "FPR": float(fpr),
        "TNR": float(tnr),
    }

    if verbose:
        print(
            f"H={H}h | AUROC={metrics['AUROC']:.3f}  AUPRC={metrics['AUPRC']:.3f}  "
            f"Brier={metrics['Brier']:.3f}  MAE_temp={metrics['MAE_temp']:.2f}"
        )
        print(
            f"thr={threshold:.2f} | TP={tp} FP={fp} TN={tn} FN={fn} "
            f"(precision={precision:.3f}, recall={recall:.3f}, FPR={fpr:.3f})"
        )
        print(
            f"There is a {p_te[-1]*100:.2f}% chance of frost in the next {H} hours, "
            f"predicted temperature: {yhat_te[-1]:.2f} °C"
        )

    return calib, reg, metrics

In [ ]:
name = "80-fresnostate" # Station Name
df = datasets[name] # DataSet
tfs = [3, 6, 12, 24] # TimeFrames
# For Each TimeFrame Train/Test a Model
for tf in tfs:
    _calib, _reg, _metrics = train_model(df, H=tf, threshold=0.5, verbose=True)

# 🤝 Teacher Ensemble Distillation: Unified Student Frost & Temperature Model
- **Weighted teacher ensemble** using accuracy–correlation scores (acc / Σ corr² → softmax).
- **Student classification** model trained from blended hard labels + teacher soft probabilities.
- **Student regression** model distilled from ensemble temperature forecasts.
- Enables a **global multi-station model** with improved generalization across microclimates.


In [9]:
def _compute_teacher_weights(teachers, X_ref, y_ref, eps=1e-6):
    """
    Compute teacher weights:

        score_i = acc_i / sum_j corr(i, j)^2
        w_i = softmax(score_i)

    where:
        acc_i = accuracy of teacher i on (X_ref, y_ref)
        corr(i, j) = Pearson correlation between teacher i and j's probabilities
                     (using raw correlation, then squared before summing).
    """
    n_teachers = len(teachers)
    if n_teachers == 1:
        return np.array([1.0], dtype=float)

    # (n_samples, n_teachers) matrix of predicted probabilities for class 1 (frost)
    preds = np.column_stack([t.predict_proba(X_ref)[:, 1] for t in teachers])

    # accuracy per teacher at threshold 0.5
    hard_preds = (preds >= 0.5).astype(int)          # (n_samples, n_teachers)
    y_ref_arr = y_ref.values.reshape(-1, 1)          # (n_samples, 1)
    acc = (hard_preds == y_ref_arr).mean(axis=0)     # (n_teachers,)

    # Pearson correlation matrix between teacher outputs
    # corr_matrix[i, j] = corr(preds[:, i], preds[:, j])
    corr_matrix = np.corrcoef(preds, rowvar=False)   # (n_teachers, n_teachers)

    # we don't want self-correlation; set diagonal to 0 so it doesn't affect the sum
    np.fill_diagonal(corr_matrix, 0.0)

    # sum of squared correlations to *other* teachers
    # denom_i = Σ_j corr(i, j)^2
    sum_sq_corr = np.sum(corr_matrix ** 2, axis=1)   # (n_teachers,)

    # score = acc / (sum of squared correlations)
    score = acc / (sum_sq_corr + eps)

    # softmax normalization
    score_shifted = score - score.max()              # for numerical stability
    exp_score = np.exp(score_shifted)
    weights = exp_score / exp_score.sum()            # (n_teachers,)

    return weights


def _ensemble_probs(teachers, X, weights=None):
    """Compute ensemble frost probability for each row in X."""
    preds = np.column_stack([t.predict_proba(X)[:, 1] for t in teachers])  # (n_samples, n_teachers)

    if weights is None:
        # equal weights if none provided
        w = np.ones(preds.shape[1], dtype=float) / preds.shape[1]
    else:
        w = np.asarray(weights, dtype=float)
        w = w / w.sum()

    return preds @ w  # (n_samples, n_teachers) @ (n_teachers,) -> (n_samples,)

def _ensemble_reg_preds(regressors, X, weights=None):
    """Ensemble temperature predictions from regression teachers."""
    preds = np.column_stack([r.predict(X) for r in regressors])
    # preds shape: (n_samples, n_regressors)

    if weights is None:
        w = np.ones(preds.shape[1], dtype=float) / preds.shape[1]
    else:
        w = np.asarray(weights, dtype=float)
        w = w / w.sum()

    return preds @ w  # (n_samples, n_regressors) @ (n_regressors,) -> (n_samples,)

def train_student_from_ensemble(
    df,
    teachers,
    reg_teachers,
    H=6,
    threshold=0.5,
    teacher_weights=None,   # if None, computed via acc / sum corr^2 on calibration set
    alpha=0.7,              # KD blend for classification
    beta=0.7,               # KD blend for regression
    verbose=True,
):
    """
    Train a student LightGBM 'classifier' (as regressor on [0,1]) + regressor by distilling knowledge
    from ensembles of teacher classifiers and teacher regressors.

    teachers: list of fitted classifiers with predict_proba
    reg_teachers: list of fitted regressors with predict (same ordering as teachers)
    teacher_weights: optional list of weights; if None, computed from classifier teachers.
    alpha: blend for classification KD (1.0 -> pure ground truth, 0.0 -> pure teacher)
    beta:  blend for regression KD (1.0 -> pure ground truth, 0.0 -> pure teacher)
    """

    # --- Build features/targets as in your original pipeline ---
    X, y_frost, y_temp, feat_cols = build_xy(df, H)

    # Time-ordered split: 70% train, 15% calibration, 15% test
    n = len(X)
    i_tr, i_cal = int(n * 0.70), int(n * 0.85)

    X_tr, y_tr = X.iloc[:i_tr],        y_frost.iloc[:i_tr]
    X_cal, y_cal = X.iloc[i_tr:i_cal], y_frost.iloc[i_tr:i_cal]
    X_te,  y_te  = X.iloc[i_cal:],     y_frost.iloc[i_cal:]
    y_temp_trcal = y_temp.iloc[:i_cal]   # for regressor: train + cal
    y_temp_te    = y_temp.iloc[i_cal:]

    X_trcal = pd.concat([X_tr, X_cal])    # for regression KD features

    # ---------- 1) If no weights are provided, compute them on the calibration slice ----------
    if teacher_weights is None:
        teacher_weights = _compute_teacher_weights(teachers, X_cal, y_cal)
        if verbose:
            print("Teacher weights (acc / sum corr^2 → softmax):", teacher_weights)

    # ---------- 2) Classification KD: ensemble teacher probabilities ----------
    p_teacher_tr  = _ensemble_probs(teachers, X_tr,  teacher_weights)
    p_teacher_cal = _ensemble_probs(teachers, X_cal, teacher_weights)  # can be used later if you want calibration

    # KD targets for classification: blend hard labels + teacher soft probs
    # y_tr in {0,1}, p_teacher_tr in [0,1]
    y_tr_kd = alpha * y_tr.values + (1.0 - alpha) * p_teacher_tr

    # ---------- 3) Train student 'classifier' as a regressor on KD labels ----------
    # This model outputs a continuous value in [~0,1] which we interpret as frost probability.
    student_clf = lgb.LGBMRegressor(
        n_estimators=800,
        learning_rate=0.002,
        subsample=0.8,
        colsample_bytree=0.9,
        random_state=42,
    )
    student_clf.fit(X_tr, y_tr_kd)

    # Student test "probabilities" (regression output, clipped to [0,1])
    p_te_student = student_clf.predict(X_te)
    p_te_student = np.clip(p_te_student, 0.0, 1.0)

    # Hard predictions at chosen threshold
    y_pred = (p_te_student >= threshold).astype(int)

    # ---------- 4) Regression KD: ensemble teacher temperature predictions ----------
    # Teacher temp ensemble for train+cal window
    temp_teacher_trcal = _ensemble_reg_preds(reg_teachers, X_trcal, teacher_weights)

    # KD targets: blend hard temps + teacher temps
    y_temp_trcal_kd = beta * y_temp_trcal.values + (1.0 - beta) * temp_teacher_trcal

    # Train student regressor on KD temps
    student_reg = lgb.LGBMRegressor(
        n_estimators=400,
        learning_rate=0.04,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
    )
    student_reg.fit(X_trcal, y_temp_trcal_kd)

    # Student temp predictions on test
    yhat_te = student_reg.predict(X_te)

    # ---------- 5) Metrics ----------
    tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    fpr       = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    tnr       = tn / (fp + tn) if (fp + tn) > 0 else 0.0

    metrics = {
        "AUROC":    float(roc_auc_score(y_te, p_te_student)),
        "AUPRC":    float(average_precision_score(y_te, p_te_student)),
        "Brier":    float(brier_score_loss(y_te, p_te_student)),
        "MAE_temp": float(mean_absolute_error(y_temp_te, yhat_te)),
        "threshold": float(threshold),
        "TP": int(tp),
        "FP": int(fp),
        "TN": int(tn),
        "FN": int(fn),
        "precision": float(precision),
        "recall": float(recall),
        "FPR": float(fpr),
        "TNR": float(tnr),
        "alpha": float(alpha),
        "beta": float(beta),
        "teacher_weights": teacher_weights.tolist(),
    }

    if verbose:
        print(
            f"[STUDENT] H={H}h | AUROC={metrics['AUROC']:.3f}  "
            f"AUPRC={metrics['AUPRC']:.3f}  Brier={metrics['Brier']:.3f}  "
            f"MAE_temp={metrics['MAE_temp']:.2f}"
        )
        print(
            f"thr={threshold:.2f} | TP={tp} FP={fp} TN={tn} FN={fn} "
            f"(precision={precision:.3f}, recall={recall:.3f}, FPR={fpr:.3f})"
        )
        print(
            f"There is a {p_te_student[-1]*100:.2f}% chance of frost in the next {H} hours, "
            f"predicted temperature: {yhat_te[-1]:.2f} °C"
        )

    return student_clf, student_reg, metrics


In [11]:
tfs = [3,] # TimeFrames
teachers = [] # Teachers (Classificaion)
reg_teachers = [] # Teachers (Regression)
stations = list(datasets.keys()) # Stations
df_all = pd.concat(list(datasets.values()), ignore_index=True) # DataFrame with All Stations
for tf in tfs:
    for station in stations:
        print(f"\nTraining {station}..." )
        calib, reg, _ = train_model(datasets[station], tf, threshold=0.5, verbose=True)
        teachers.append(calib); reg_teachers.append(reg)
    print(f"\nTraining Student from Teachers...")
    calib_student, student_reg, metrics = (
        # Alpha and Beta are best at 0.7
        train_student_from_ensemble(df_all, teachers, reg_teachers, tf, 
                                    threshold=0.5,alpha=0.7, beta=0.7, verbose=True)
    )


Training 105-westlands...
H=3h | AUROC=0.989  AUPRC=0.806  Brier=0.004  MAE_temp=0.99
thr=0.50 | TP=149 FP=64 TN=19237 FN=36 (precision=0.700, recall=0.805, FPR=0.003)
There is a 0.02% chance of frost in the next 3 hours, predicted temperature: 19.73 °C

Training 124-panoche...
H=3h | AUROC=0.980  AUPRC=0.642  Brier=0.003  MAE_temp=1.09
thr=0.50 | TP=64 FP=60 TN=18092 FN=19 (precision=0.516, recall=0.771, FPR=0.003)
There is a 0.01% chance of frost in the next 3 hours, predicted temperature: 17.96 °C

Training 125-arvinedison...
H=3h | AUROC=0.998  AUPRC=0.590  Brier=0.002  MAE_temp=1.07
thr=0.50 | TP=4 FP=2 TN=18448 FN=64 (precision=0.667, recall=0.059, FPR=0.000)
There is a 0.02% chance of frost in the next 3 hours, predicted temperature: 19.82 °C

Training 131-fairoaks...
H=3h | AUROC=0.999  AUPRC=0.768  Brier=0.001  MAE_temp=0.99
thr=0.50 | TP=23 FP=7 TN=19283 FN=9 (precision=0.767, recall=0.719, FPR=0.000)
There is a 0.00% chance of frost in the next 3 hours, predicted temperatur

In [12]:
tf = 3 # TimeFrames
stations = list(datasets.keys()) # Stations
df_all = pd.concat(list(datasets.values()), ignore_index=True) # DataFrame with All Stations
print(f"\n Training General Model...")
calib, reg, _ = train_model(df_all, tf, threshold=0.5, verbose=True)


 Training General Model...
H=3h | AUROC=0.995  AUPRC=0.714  Brier=0.005  MAE_temp=1.00
thr=0.50 | TP=2703 FP=1160 TN=329550 FN=1061 (precision=0.700, recall=0.718, FPR=0.004)
There is a 0.00% chance of frost in the next 3 hours, predicted temperature: 20.29 °C


In [13]:
import joblib
import os

MODEL_DIR = "new_models"
# Create directory for saving
os.makedirs(MODEL_DIR, exist_ok=True)

# Save teachers
for i, (clf, reg) in enumerate(zip(teachers, reg_teachers)):
    joblib.dump(clf, f"{MODEL_DIR}/teacher_clf_{i}.pkl")
    joblib.dump(reg, f"{MODEL_DIR}/teacher_reg_{i}.pkl")

# Save student models
joblib.dump(calib_student, f"{MODEL_DIR}/student_clf.pkl")
joblib.dump(student_reg, f"{MODEL_DIR}/student_reg.pkl")

print("Models saved successfully.")

Models saved successfully.
